In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import pmdarima as pm
from pmdarima.utils import diff
from sklearn.metrics import mean_squared_error
from pmdarima.metrics import smape
import pickle
import sklearn

# Importing and preparing zri and acs data for merging

In [2]:
zri = pd.read_csv('./../data/zri_multifamily_v2.csv')

In [3]:
zri['zip'] = zri['zip'].map(lambda x: str(x))

In [4]:
zri['zip'].map(lambda x: len(x)).value_counts()

5    79272
4    14472
Name: zip, dtype: int64

In [5]:
zri['zip'] = zri['zip'].map(lambda x: '0' + x if len(x)<5 else x)

In [6]:
zri['zip'].map(lambda x: len(x)).value_counts()

5    93744
Name: zip, dtype: int64

In [7]:
zri['year']=zri['year-month'].map(lambda x: int(x[0:4]))

In [8]:
zri['month']=zri['year-month'].map(lambda x: int(x[5:]))

In [9]:
years = range(2014,2020)

In [10]:
months = range(1, 13)

In [11]:
datetime_col = [datetime(year=i, month=j, day=1) for i in years for j in months]

In [12]:
datetime_col = datetime_col*1302

In [13]:
zri['datetime'] = datetime_col

In [14]:
zri.drop(columns = 'year-month', inplace = True)

In [15]:
zri

,zip,City,State,Metro,CountyName,zri,year,month,datetime
0,01013,Chicopee,MA,Springfield,Hampden County,928.0,2014,1,2014-01-01
1,01013,Chicopee,MA,Springfield,Hampden County,931.0,2014,2,2014-02-01
2,01013,Chicopee,MA,Springfield,Hampden County,934.0,2014,3,2014-03-01
3,01013,Chicopee,MA,Springfield,Hampden County,929.0,2014,4,2014-04-01
4,01013,Chicopee,MA,Springfield,Hampden County,929.0,2014,5,2014-05-01
...,...,...,...,...,...,...,...,...,...
93739,99654,Wasilla,AK,Anchorage,Matanuska Susitna Borough,1277.0,2019,8,2019-08-01
93740,99654,Wasilla,AK,Anchorage,Matanuska Susitna Borough,1271.0,2019,9,2019-09-01
93741,99654,Wasilla,AK,Anchorage,Matanuska Susitna Borough,1299.0,2019,10,2019-10-01
93742,99654,Wasilla,AK,Anchorage,Matanuska Susitna Borough,1261.5,2019,11,2019-11-01


In [16]:
acs = pd.read_csv('./../data/acs_engineered_features.csv')

In [17]:
acs['zip'] = acs['zip'].map(lambda x: str(x))

In [18]:
acs['zip'].map(lambda x: len(x)).value_counts()

5    83482
4     9170
Name: zip, dtype: int64

In [19]:
acs['zip'] = acs['zip'].map(lambda x: '0' + x if len(x)<5 else x)

In [20]:
acs['zip'].map(lambda x: len(x)).value_counts()

5    92652
Name: zip, dtype: int64

In [21]:
acs.drop(columns = 'census_period', axis = 1, inplace = True)

In [22]:
acs

,zip,year_usable,percent_white,percent_black,percent_asian,percent_hispanic,percent_native_am,percent_other_race,percent_0_17,percent_18_39,...,median_building_age,income_per_capita,poverty_rate,total_pop,percent_workforce_unemployed,percent_work_from_home,median_age,percent_female,gini_index,percent_not_us_citizen
0,01013,2013,0.729579,0.020723,0.013756,0.218793,0.000000,0.005002,0.230494,0.311375,...,62.0,20433.0,0.190522,22391,0.104113,0.008853,36.4,0.510250,0.4210,0.055960
1,01013,2014,0.714417,0.023463,0.013473,0.231235,0.000000,0.004491,0.224682,0.312437,...,62.0,20940.0,0.201543,21822,0.104320,0.014770,36.6,0.501650,0.4179,0.057190
2,01013,2015,0.720119,0.027857,0.016777,0.217971,0.001357,0.005065,0.217248,0.335730,...,66.0,20889.0,0.173591,22113,0.113411,0.012946,35.5,0.506851,0.4110,0.056483
3,01013,2016,0.713799,0.017445,0.016638,0.237499,0.000538,0.002197,0.221131,0.328625,...,72.0,21371.0,0.155696,22299,0.115569,0.013701,35.8,0.512669,0.4061,0.050182
4,01013,2017,0.703894,0.022238,0.018442,0.241627,0.001116,0.001652,0.222336,0.338573,...,73.0,21477.0,0.169816,22394,0.114075,0.020243,34.9,0.514200,0.4117,0.050013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92647,99901,2015,0.653872,0.002271,0.063146,0.044026,0.142407,0.002710,0.233756,0.279833,...,38.0,31563.0,0.101846,13651,0.086316,0.021073,38.7,0.478353,0.3981,0.025493
92648,99901,2016,0.654374,0.004016,0.061487,0.046444,0.143932,0.000000,0.232584,0.279977,...,38.0,31487.0,0.114447,13694,0.085095,0.022509,38.8,0.481233,0.4057,0.023952
92649,99901,2017,0.649705,0.004228,0.073631,0.048334,0.135379,0.001604,0.230079,0.285121,...,40.0,32010.0,0.121194,13717,0.078951,0.023050,38.7,0.485602,0.4110,0.025953
92650,99901,2018,0.645762,0.004947,0.076319,0.046126,0.129138,0.005384,0.225609,0.284103,...,40.0,32671.0,0.108124,13745,0.075428,0.026582,39.2,0.485922,0.4158,0.035504


In [23]:
zri.drop(columns = ['City', 'State', 'Metro', 'CountyName'], inplace = True)

In [24]:
merged = pd.merge(zri, acs, left_on = ['zip','year'], right_on = ['zip','year_usable'], how = 'left')

In [25]:
null_check = {}
for col in merged.columns:
    if merged[col].isnull().any():
        null_check[col] = merged[col].isnull().sum()
null_check

{'year_usable': 72,
 'percent_white': 72,
 'percent_black': 72,
 'percent_asian': 72,
 'percent_hispanic': 72,
 'percent_native_am': 72,
 'percent_other_race': 72,
 'percent_0_17': 72,
 'percent_18_39': 72,
 'percent_40_64': 72,
 'percent_65+': 72,
 'percent_rental_units_vacant': 72,
 'percent_rental_units_occupied': 72,
 'percent_graduate_deg': 72,
 'percent_bachelors': 72,
 'percent_associates': 72,
 'percent_highschool': 72,
 'percent_less_highschool': 72,
 'percent_commute_public_transport': 72,
 'percent_commute_less_30': 72,
 'percent_commute_30_to_59': 72,
 'percent_commute_60_to_89': 72,
 'percent_commute_90_more': 72,
 'percent_new_city': 72,
 'percent_new_unit': 72,
 'percent_units_owner_occupied': 72,
 'median_building_age': 72,
 'income_per_capita': 72,
 'poverty_rate': 72,
 'total_pop': 72,
 'percent_workforce_unemployed': 72,
 'percent_work_from_home': 72,
 'median_age': 72,
 'percent_female': 72,
 'gini_index': 72,
 'percent_not_us_citizen': 72}

In [26]:
merged= merged[merged['zip']!= '11249']

In [27]:
null_check = {}
for col in merged.columns:
    if merged[col].isnull().any():
        null_check[col] = merged[col].isnull().sum()
null_check

{}

In [28]:
merged['zip'].nunique()

1301

In [29]:
df = merged.drop(columns = ['year_usable'])

# Using CPI rent index to adjust for the value of the dollar across the timeframe

In [30]:
inf = pd.read_csv('./rent_inflation_cpi.csv')
inf.rename(columns = {'DATE':'Date', 'CUUR0000SEHA':'cpi_rent'},inplace=True)
inf['year'] = inf['Date'].map(lambda x: int(x[0:4]))
inf['month'] = inf['Date'].map(lambda x: int(x[5:7]))
inf = inf.loc[:, ['year', 'month', 'cpi_rent']]

In [31]:
inf

,year,month,cpi_rent
0,2014,1,272.317
1,2014,2,272.733
2,2014,3,273.486
3,2014,4,274.100
4,2014,5,274.710
...,...,...,...
68,2019,9,333.834
69,2019,10,334.680
70,2019,11,335.819
71,2019,12,336.789


In [32]:
# dxy = pd.read_csv('./DX-Y.NYB.csv')
# dxy['year'] = dxy['Date'].map(lambda x: int(x[0:4]))
# dxy['month'] = dxy['Date'].map(lambda x: int(x[5:7]))
# dxy = dxy.loc[:, ['year', 'month', 'Adj Close']]
# dxy.rename(columns = {'Adj Close' : 'pct_jan_2020_usd'}, inplace = True)
# dxy['pct_jan_2020_usd'] = dxy['pct_jan_2020_usd'].interpolate(method='linear')
# dxy = dxy.groupby(['year', 'month']).agg(pct_jan_2020_usd = ('pct_jan_2020_usd', 'mean'))
# dxy = dxy.reset_index(drop=False)
# dxy['pct_jan_2020_usd'] = dxy['pct_jan_2020_usd'].map(lambda x: x/float(dxy[dxy['year']==2020]['pct_jan_2020_usd']))
# dxy = dxy[dxy['year']!=2020]
# dxy

In [33]:
df

,zip,zri,year,month,datetime,percent_white,percent_black,percent_asian,percent_hispanic,percent_native_am,...,median_building_age,income_per_capita,poverty_rate,total_pop,percent_workforce_unemployed,percent_work_from_home,median_age,percent_female,gini_index,percent_not_us_citizen
0,01013,928.0,2014,1,2014-01-01,0.714417,0.023463,0.013473,0.231235,0.000000,...,62.0,20940.0,0.201543,21822.0,0.104320,0.014770,36.6,0.501650,0.4179,0.057190
1,01013,931.0,2014,2,2014-02-01,0.714417,0.023463,0.013473,0.231235,0.000000,...,62.0,20940.0,0.201543,21822.0,0.104320,0.014770,36.6,0.501650,0.4179,0.057190
2,01013,934.0,2014,3,2014-03-01,0.714417,0.023463,0.013473,0.231235,0.000000,...,62.0,20940.0,0.201543,21822.0,0.104320,0.014770,36.6,0.501650,0.4179,0.057190
3,01013,929.0,2014,4,2014-04-01,0.714417,0.023463,0.013473,0.231235,0.000000,...,62.0,20940.0,0.201543,21822.0,0.104320,0.014770,36.6,0.501650,0.4179,0.057190
4,01013,929.0,2014,5,2014-05-01,0.714417,0.023463,0.013473,0.231235,0.000000,...,62.0,20940.0,0.201543,21822.0,0.104320,0.014770,36.6,0.501650,0.4179,0.057190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93739,99654,1277.0,2019,8,2019-08-01,0.794726,0.008443,0.014660,0.050576,0.049059,...,23.0,30300.0,0.095106,60641.0,0.102037,0.050081,33.6,0.476311,0.4004,0.018997
93740,99654,1271.0,2019,9,2019-09-01,0.794726,0.008443,0.014660,0.050576,0.049059,...,23.0,30300.0,0.095106,60641.0,0.102037,0.050081,33.6,0.476311,0.4004,0.018997
93741,99654,1299.0,2019,10,2019-10-01,0.794726,0.008443,0.014660,0.050576,0.049059,...,23.0,30300.0,0.095106,60641.0,0.102037,0.050081,33.6,0.476311,0.4004,0.018997
93742,99654,1261.5,2019,11,2019-11-01,0.794726,0.008443,0.014660,0.050576,0.049059,...,23.0,30300.0,0.095106,60641.0,0.102037,0.050081,33.6,0.476311,0.4004,0.018997


In [34]:
df = pd.merge(df, inf, on = ['year', 'month'], how = 'left')

In [35]:
df['zri_adj'] = df['zri']/df['cpi_rent']*100

In [36]:
df.drop(columns = 'cpi_rent', inplace = True)

In [37]:
df = df.set_index('datetime')

In [38]:
df

,zip,zri,year,month,percent_white,percent_black,percent_asian,percent_hispanic,percent_native_am,percent_other_race,...,income_per_capita,poverty_rate,total_pop,percent_workforce_unemployed,percent_work_from_home,median_age,percent_female,gini_index,percent_not_us_citizen,zri_adj
datetime,,,,,,,,,,,,,,,,,,,,,
2014-01-01,01013,928.0,2014,1,0.714417,0.023463,0.013473,0.231235,0.000000,0.004491,...,20940.0,0.201543,21822.0,0.104320,0.014770,36.6,0.501650,0.4179,0.057190,340.779312
2014-02-01,01013,931.0,2014,2,0.714417,0.023463,0.013473,0.231235,0.000000,0.004491,...,20940.0,0.201543,21822.0,0.104320,0.014770,36.6,0.501650,0.4179,0.057190,341.359498
2014-03-01,01013,934.0,2014,3,0.714417,0.023463,0.013473,0.231235,0.000000,0.004491,...,20940.0,0.201543,21822.0,0.104320,0.014770,36.6,0.501650,0.4179,0.057190,341.516568
2014-04-01,01013,929.0,2014,4,0.714417,0.023463,0.013473,0.231235,0.000000,0.004491,...,20940.0,0.201543,21822.0,0.104320,0.014770,36.6,0.501650,0.4179,0.057190,338.927399
2014-05-01,01013,929.0,2014,5,0.714417,0.023463,0.013473,0.231235,0.000000,0.004491,...,20940.0,0.201543,21822.0,0.104320,0.014770,36.6,0.501650,0.4179,0.057190,338.174803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08-01,99654,1277.0,2019,8,0.794726,0.008443,0.014660,0.050576,0.049059,0.003051,...,30300.0,0.095106,60641.0,0.102037,0.050081,33.6,0.476311,0.4004,0.018997,383.900817
2019-09-01,99654,1271.0,2019,9,0.794726,0.008443,0.014660,0.050576,0.049059,0.003051,...,30300.0,0.095106,60641.0,0.102037,0.050081,33.6,0.476311,0.4004,0.018997,380.728146
2019-10-01,99654,1299.0,2019,10,0.794726,0.008443,0.014660,0.050576,0.049059,0.003051,...,30300.0,0.095106,60641.0,0.102037,0.050081,33.6,0.476311,0.4004,0.018997,388.131947


# Train-test splitting and modeling

In [39]:
arima = pm.arima.auto_arima

In [40]:
def forecast(number_steps, model):
    pred = []
    conf = []
    
    for i in range(number_steps):
        fc, conf_int = model.predict(n_periods=1, return_conf_int=True)
        fc = fc.tolist()[0]
        conf_int = np.asarray(conf_int).tolist()[0]
        pred.append(fc)
        conf.append(conf_int)

        # Updates the existing model with the prediction it just made
        model.update(fc)
    return pred, conf

In [41]:
zips = list(df['zip'].unique())

test = df[df['year']>=2019].drop(columns = ['year', 'month'])
train = df[df['year']<2019].drop(columns = ['year', 'month'])

y_train = train[['zip','zri', 'zri_adj']]
y_test = test[['zip','zri', 'zri_adj']]

x_train = train.drop(columns = ['zri', 'zri_adj'])
x_test = test.drop(columns = ['zri', 'zri_adj'])

In [42]:
order_dict = {}
rmse_dict = {}
resid_dict = {}
pred_dict = {}
conf_int_dict = {}

In [43]:
start = datetime.now()
for item in zips:
    curr_train = y_train[y_train['zip']==item]
    curr_test = y_test[y_test['zip']==item]
    
    curr_model = arima(curr_train['zri'], start_p=0, start_q=0, m=12)
    order_dict[item] = (curr_model.order, curr_model.seasonal_order)
    
    pred_dict[item], conf_int_dict[item] = forecast(curr_test.shape[0], curr_model)
    resid_dict[item] = pred_dict[item] - curr_test['zri']
    
    rmse_dict[item] = np.sqrt(sum(resid_dict[item]**2)/len(resid_dict[item]))
elapsed = datetime.now()-start
print(elapsed)

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\_auto_solvers.py:522: ModelFitWarning: Error fitting  ARIMA(2,0,0)(0,0,0)[12]           (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\_auto_solvers.py", line 506, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 482, in fit
    self._fit(y, X, **fit_args)
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 403, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 393, in _fit_wrapper
    return arima, arima.fit(start_params=start_params,
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\statespace\mlemodel.py", line 6

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\_auto_solvers.py:522: ModelFitWarning: Error fitting  ARIMA(3,1,2)(1,0,1)[12]           (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\_auto_solvers.py", line 506, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 482, in fit
    self._fit(y, X, **fit_args)
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 403, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 393, in _fit_wrapper
    return arima, arima.fit(start_params=start_params,
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\statespace\mlemodel.py", line 6

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\_auto_solvers.py:522: ModelFitWarning: Error fitting  ARIMA(1,0,1)(1,0,0)[12] intercept (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\_auto_solvers.py", line 506, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 482, in fit
    self._fit(y, X, **fit_args)
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 403, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 393, in _fit_wrapper
    return arima, arima.fit(start_params=start_params,
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\statespace\mlemodel.py", line 6

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\_auto_solvers.py:522: ModelFitWarning: Error fitting  ARIMA(3,0,0)(1,0,0)[12] intercept (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\_auto_solvers.py", line 506, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 482, in fit
    self._fit(y, X, **fit_args)
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 403, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 393, in _fit_wrapper
    return arima, arima.fit(start_params=start_params,
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\statespace\mlemodel.py", line 6

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\_auto_solvers.py:522: ModelFitWarning: Error fitting  ARIMA(2,0,0)(0,0,0)[12]           (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\_auto_solvers.py", line 506, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 482, in fit
    self._fit(y, X, **fit_args)
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 403, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 393, in _fit_wrapper
    return arima, arima.fit(start_params=start_params,
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\statespace\mlemodel.py", line 6

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\_auto_solvers.py:522: ModelFitWarning: Error fitting  ARIMA(2,0,1)(1,0,0)[12] intercept (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\_auto_solvers.py", line 506, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 482, in fit
    self._fit(y, X, **fit_args)
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 403, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 393, in _fit_wrapper
    return arima, arima.fit(start_params=start_params,
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\statespace\mlemodel.py", line 6

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\_auto_solvers.py:522: ModelFitWarning: Error fitting  ARIMA(2,0,0)(0,0,0)[12]           (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\_auto_solvers.py", line 506, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 482, in fit
    self._fit(y, X, **fit_args)
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 403, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 393, in _fit_wrapper
    return arima, arima.fit(start_params=start_params,
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\statespace\mlemodel.py", line 6

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\_auto_solvers.py:522: ModelFitWarning: Error fitting  ARIMA(3,1,1)(0,0,0)[12] intercept (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\_auto_solvers.py", line 506, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 482, in fit
    self._fit(y, X, **fit_args)
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 403, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 393, in _fit_wrapper
    return arima, arima.fit(start_params=start_params,
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\statespace\mlemodel.py", line 6

0:58:13.027645


# Seeing whether modeling on deflated or normal zri data gives better accuracy

In [44]:
order_dict_adj = {}
rmse_dict_adj = {}
resid_dict_adj = {}
pred_dict_adj = {}
conf_int_dict_adj = {}

In [45]:
start = datetime.now()
for item in zips:
    curr_train = y_train[y_train['zip']==item]
    curr_test = y_test[y_test['zip']==item]
    
    curr_model = arima(curr_train['zri_adj'], start_p=0, start_q=0, m=12)
    order_dict_adj[item] = (curr_model.order, curr_model.seasonal_order)
    
    pred_dict_adj[item], conf_int_dict_adj[item] = forecast(curr_test.shape[0], curr_model)
    resid_dict_adj[item] = pred_dict_adj[item] - curr_test['zri_adj']
    
    rmse_dict_adj[item] = np.sqrt(sum(resid_dict_adj[item]**2)/len(resid_dict_adj[item]))
elapsed = datetime.now()-start
print(elapsed)

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\_auto_solvers.py:522: ModelFitWarning: Error fitting  ARIMA(2,0,2)(1,0,0)[12] intercept (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\_auto_solvers.py", line 506, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 482, in fit
    self._fit(y, X, **fit_args)
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 403, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 393, in _fit_wrapper
    return arima, arima.fit(start_params=start_params,
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\statespace\mlemodel.py", line 6

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\_auto_solvers.py:522: ModelFitWarning: Error fitting  ARIMA(1,1,0)(1,0,1)[12] intercept (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\_auto_solvers.py", line 506, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 482, in fit
    self._fit(y, X, **fit_args)
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 403, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 393, in _fit_wrapper
    return arima, arima.fit(start_params=start_params,
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\statespace\mlemodel.py", line 6

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\_auto_solvers.py:522: ModelFitWarning: Error fitting  ARIMA(2,0,0)(0,0,1)[12]           (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\_auto_solvers.py", line 506, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 482, in fit
    self._fit(y, X, **fit_args)
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 403, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 393, in _fit_wrapper
    return arima, arima.fit(start_params=start_params,
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\statespace\mlemodel.py", line 6

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\_auto_solvers.py:522: ModelFitWarning: Error fitting  ARIMA(3,0,0)(1,0,0)[12] intercept (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\_auto_solvers.py", line 506, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 482, in fit
    self._fit(y, X, **fit_args)
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 403, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 393, in _fit_wrapper
    return arima, arima.fit(start_params=start_params,
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\statespace\mlemodel.py", line 6

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\_auto_solvers.py:522: ModelFitWarning: Error fitting  ARIMA(2,0,0)(0,0,0)[12]           (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\_auto_solvers.py", line 506, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 482, in fit
    self._fit(y, X, **fit_args)
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 403, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 393, in _fit_wrapper
    return arima, arima.fit(start_params=start_params,
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\statespace\mlemodel.py", line 6

...

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\_auto_solvers.py:522: ModelFitWarning: Error fitting  ARIMA(1,0,0)(1,0,0)[12] intercept (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\_auto_solvers.py", line 506, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 482, in fit
    self._fit(y, X, **fit_args)
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 403, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 393, in _fit_wrapper
    return arima, arima.fit(start_params=start_params,
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\statespace\mlemodel.py", line 6

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\_auto_solvers.py:522: ModelFitWarning: Error fitting  ARIMA(2,0,0)(1,0,0)[12] intercept (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\_auto_solvers.py", line 506, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 482, in fit
    self._fit(y, X, **fit_args)
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 403, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 393, in _fit_wrapper
    return arima, arima.fit(start_params=start_params,
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\statespace\mlemodel.py", line 6

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\_auto_solvers.py:522: ModelFitWarning: Error fitting  ARIMA(2,0,0)(0,0,0)[12]           (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\_auto_solvers.py", line 506, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 482, in fit
    self._fit(y, X, **fit_args)
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 403, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 393, in _fit_wrapper
    return arima, arima.fit(start_params=start_params,
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\statespace\mlemodel.py", line 6

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\_auto_solvers.py:522: ModelFitWarning: Error fitting  ARIMA(2,0,0)(0,0,0)[12]           (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\_auto_solvers.py", line 506, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 482, in fit
    self._fit(y, X, **fit_args)
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 403, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\pmdarima\arima\arima.py", line 393, in _fit_wrapper
    return arima, arima.fit(start_params=start_params,
  File "C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\statespace\mlemodel.py", line 6

1:02:25.637029


In [127]:
fig = go.Figure()
fig.add_trace(go.Line(x = curr_train.index, y = curr_train['zri'], mode = 'lines'))
fig.add_trace(go.Line(x = curr_test.index, y = curr_test['zri'], mode = 'lines'))
fig.add_trace(go.Line(x = curr_test.index, y = pred_dict[curr_train['zip'][0]], mode = 'lines'))
# fig.add_trace(go.Line(x = curr_test.index, y = np.asarray(conf)[:,0], mode = 'lines'))
# fig.add_trace(go.Line(x = curr_test.index, y = np.asarray(conf)[:,1], mode = 'lines', fill = 'tonexty'))

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [52]:
fig = go.Figure()
fig.add_trace(go.Line(x = curr_train.index, y = curr_train['zri_adj'], mode = 'lines'))
fig.add_trace(go.Line(x = curr_test.index, y = curr_test['zri_adj'], mode = 'lines'))
fig.add_trace(go.Line(x = curr_test.index, y = pred_dict_adj[curr_train['zip'][0]], mode = 'lines'))
# fig.add_trace(go.Line(x = curr_test.index, y = np.asarray(conf)[:,0], mode = 'lines'))
# fig.add_trace(go.Line(x = curr_test.index, y = np.asarray(conf)[:,1], mode = 'lines', fill = 'tonexty'))

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [98]:
resid_df_adj = pd.DataFrame(resid_dict_adj).reset_index(drop=True)

In [99]:
for col in resid_df_adj.columns:
    resid_df_adj[col] = resid_df_adj[col] / 100 * inf.loc[60:71, 'cpi_rent'].reset_index(drop=True)

In [104]:
resid_df_adj = resid_df_adj.T

In [116]:
resid_df = pd.DataFrame(resid_dict).reset_index(drop=True)

In [118]:
for col in resid_df.columns:
    resid_df[col] = resid_df[col] / inf.loc[60:71, 'cpi_rent'].reset_index(drop=True) * 100

In [121]:
resid_df = resid_df.T

In [57]:
np.mean(list(rmse_dict.values())) # RMSE from inflated model residuals

50.1395887293849

In [112]:
np.mean(np.sqrt((resid_df_adj**2).sum(axis = 1)/12)) # RMSE from deflated model residuals, converted back to inflated

52.34616016658373

In [122]:
np.mean(np.sqrt((resid_df**2).sum(axis = 1)/12)) # RMSE from inflated model residuals, converted to deflated

15.065473493598674

In [124]:
np.mean(list(rmse_dict_adj.values())) # RMSE from deflated model residuals

15.726478778012323

SARIMA on non-deflated values performs better

# Save residuals, predictions, conf intervals

In [126]:
# resid_df.to_csv('./../data/sarima_residuals.csv', index = True)

In [132]:
pred_df = pd.DataFrame(pred_dict).reset_index(drop=True).T

In [134]:
pred_df

,0,1,2,3,4,5,6,7,8,9,10,11
01013,1104.920592,1102.146265,1100.259016,1098.974887,1098.100926,1097.505982,1097.100885,1096.824995,1096.637061,1096.509016,1096.421757,1096.362281
01020,1127.026942,1128.024523,1131.045334,1133.237079,1136.705090,1139.229452,1138.907327,1140.064649,1140.466025,1139.881307,1141.147032,1143.190715
01040,1096.846344,1094.689110,1093.568453,1092.986131,1092.683470,1092.526124,1092.444305,1092.401750,1092.379611,1092.368092,1092.362097,1092.358976
01085,1158.325647,1156.171994,1157.061113,1155.927067,1158.926705,1162.865122,1166.788937,1168.751911,1169.733276,1167.103253,1160.846342,1161.521173
01104,1113.898454,1112.625400,1112.259544,1112.248087,1112.327758,1112.401122,1112.446524,1112.468710,1112.477203,1112.479271,1112.479014,1112.478277
...,...,...,...,...,...,...,...,...,...,...,...,...
99207,1036.648159,1042.114784,1045.813097,1049.755897,1054.453601,1059.671420,1065.061327,1070.422227,1075.705446,1080.936319,1086.150457,1091.367992
99501,1268.754808,1265.871955,1261.152069,1257.078671,1254.981262,1254.946186,1256.236525,1257.874596,1259.104666,1259.602229,1259.441861,1258.923719
99504,1442.049705,1446.440504,1448.470475,1448.320175,1446.722280,1444.582003,1442.669788,1441.448682,1441.042284,1441.307412,1441.958268,1442.691230
99508,1283.450675,1285.385081,1284.815646,1282.500118,1279.540278,1276.972511,1275.473534,1275.242401,1276.052716,1277.419404,1278.803954,1279.787763


In [135]:
resid_df

,0,1,2,3,4,5,6,7,8,9,10,11
01013,1.818380,1.270493,-1.752903,-4.571378,-5.434947,-9.222502,-12.333685,-15.384594,-17.482623,-23.452547,-19.230074,-17.410818
01020,3.693812,2.765281,0.013842,-0.840616,-0.089548,-3.559843,-8.471728,-11.705022,-14.238806,-21.847345,-20.205220,-18.352525
01040,-4.654114,-9.287819,-10.818363,-10.348691,-9.205433,-11.635902,-13.436376,-15.812460,-16.960642,-21.403104,-12.994471,-9.097988
01085,-4.199779,-6.994925,-8.835951,-8.845415,-10.042509,-10.928503,-10.618375,-9.694650,-9.365950,-7.438971,-9.128030,-8.752907
01104,0.275940,-1.034040,-3.890061,-5.096755,-5.366071,-6.834724,-8.610689,-9.779788,-12.438157,-19.577127,-22.488598,-23.908656
...,...,...,...,...,...,...,...,...,...,...,...,...
99207,2.348965,0.035172,-3.721044,-8.593244,-11.097096,-11.591959,-11.139360,-9.192507,-6.079235,0.578558,-4.124109,-8.798390
99501,7.910026,9.766158,11.038362,11.585403,10.014563,8.754381,7.911981,6.576097,3.925504,-6.094709,-7.610689,-9.227226
99504,8.000597,10.246791,9.914255,8.920638,7.203129,6.527184,6.534819,7.650564,7.201868,3.378574,-1.203545,5.549834
99508,8.430875,10.536227,9.103653,7.454140,4.415069,1.806305,0.444364,-1.129636,-1.781509,-3.758992,-3.036173,-1.844549


In [136]:
# pred_df.to_csv('./../data/sarima_predictions.csv', index = True)

In [149]:
conf_df = pd.DataFrame(conf_int_dict).T

In [150]:
for col in conf_df.columns:
    conf_df[f'{col}_lower_bound'] = conf_df[col].map(lambda x: x[1])
    conf_df[f'{col}_upper_bound'] = conf_df[col].map(lambda x: x[0])

In [153]:
conf_df = conf_df.drop(columns = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])

In [155]:
# conf_df.to_csv('./../data/sarima_conf_ints.csv', index = True)

Exogenous features had basically 0 predictive power in modeling residuals, residuals seem random.

Possible that autoregressive features capture all the information of the exogenous features since each arima was zip-code specific.

# Creating average arima and running it on all zip codes

In [184]:
order_sum = pd.Series([0, 0, 0])
seasonal_sum = pd.Series([0,0,0])

In [185]:
for item in order_dict.values():
    curr, curr_seas = item
    curr = pd.Series(curr)
    curr_seas = pd.Series(curr_seas)
    order_sum += curr
    seasonal_sum += curr_seas

In [189]:
order_sum.index = ['p', 'd', 'q']

In [191]:
seasonal_sum.index = ['P', 'D', 'Q']

In [194]:
order_sum/len(zips)

p    1.416603
d    1.017679
q    0.385857
dtype: float64

In [193]:
seasonal_sum/len(zips)

P    0.199846
D    0.000769
Q    0.116065
dtype: float64

In [180]:
# average order is (1, 1, 0) (0, 0, 0, 12)

In [198]:
avg_model = pm.arima.arima.ARIMA((1,1,0), seasonal_order = (0,0,0,12))

In [207]:
resid_dict_avg = {}
pred_dict_avg = {}
conf_int_dict_avg = {}

In [272]:
start = datetime.now()
for item in zips:
    curr_train = y_train[y_train['zip']==item]
    curr_test = y_test[y_test['zip']==item]
    
    curr_model = pm.arima.arima.ARIMA((1,1,0), seasonal_order = (0,0,0,12))
    curr_model.fit(curr_train['zri'])
    
    pred_dict_avg[item], conf_int_dict_avg[item] = forecast(curr_test.shape[0], curr_model)
    resid_dict_avg[item] = pred_dict[item] - curr_test['zri']
elapsed = datetime.now()-start
print(elapsed)

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelih

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelih

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelih

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelih

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelih

...

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelih

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelih

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelih

0:05:01.865433


C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



In [273]:
resid_avg_df = (pd.DataFrame(resid_dict_avg)).T

In [274]:
resid_avg_df.columns = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [275]:
# resid_avg_df.to_csv('./../data/sarima_avg_residuals.csv', index = True)

In [278]:
resid_avg_df

,1,2,3,4,5,6,7,8,9,10,11,12
01013,7.257261,7.567699,0.172512,-6.406482,-6.470685,-16.210117,-23.744608,-31.149742,-35.473207,-52.745099,-35.984414,-27.203147
01020,12.026942,9.024523,0.045334,-2.762921,-0.294910,-11.770548,-28.092673,-38.935351,-47.533975,-73.118693,-67.852968,-61.809285
01040,-15.153656,-30.310890,-35.431547,-34.013869,-30.316530,-38.473876,-44.555695,-52.598250,-56.620389,-71.631908,-43.637903,-30.641024
01085,-13.674353,-22.828006,-28.938887,-29.072933,-33.073295,-36.134878,-35.211063,-32.248089,-31.266724,-24.896747,-30.653658,-29.478827
01104,0.898454,-3.374600,-12.740456,-16.751913,-17.672242,-22.598878,-28.553476,-32.531290,-41.522797,-65.520729,-75.520986,-80.521723
...,...,...,...,...,...,...,...,...,...,...,...,...
99207,7.648159,0.114784,-12.186903,-28.244103,-36.546399,-38.328580,-36.938673,-30.577773,-20.294554,1.936319,-13.849543,-29.632008
99501,25.754808,31.871955,36.152069,38.078671,32.981262,28.946186,26.236525,21.874596,13.104666,-20.397771,-25.558139,-31.076281
99504,26.049705,33.440504,32.470475,29.320175,23.722280,21.582003,21.669788,25.448682,24.042284,11.307412,-4.041732,18.691230
99508,27.450675,34.385081,29.815646,24.500118,14.540278,5.972511,1.473534,-3.757599,-5.947284,-12.580596,-10.196046,-6.212237


In [279]:
np.sqrt(((resid_avg_df**2).sum().sum())/(1301*12))

63.464011388459355